In [ ]:
from dotenv import load_dotenv
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

from app.cli_args import get_database_path
from infra.db_connection import db_connect
from domain.calc_profitability import calc_balance_for_all_accounts
from app.sharpe_ratio import find_best_savings_account_by_sharpe_ratio

load_dotenv()
db_path = get_database_path()
conn = db_connect(db_path)
df = calc_balance_for_all_accounts(conn)
current_date = datetime.now()
best_account = find_best_savings_account_by_sharpe_ratio(conn, current_date)
best_account

In [ ]:
df

In [ ]:
max_balance_account = df.loc[df['balance'].idxmax()]
min_balance_account = df.loc[df['balance'].idxmin()]
print('max_balance_account', max_balance_account)
print('min_balance_account', min_balance_account)

## Account Balances Bar Chart

In [ ]:

df_sorted = df.sort_values(by='balance', ascending=False)
df_sorted

default_colors = px.colors.qualitative.Plotly
highlight_color = default_colors[1]
normal_color = default_colors[0]

colors = [highlight_color if account == best_account['account_id'] else normal_color for account in df_sorted['accountId']]

fig = go.Figure()

for account, balance, color in zip(df_sorted['accountName'], df_sorted['balance'], colors):
    fig.add_trace(go.Bar(
        x=[account],
        y=[balance],
        name=account,
        marker_color=color
    ))

fig.update_layout(
    xaxis_tickangle=-90,
    xaxis_tickfont=dict(size=10),
    height=800,
    width=1000,
    yaxis=dict(range=[df['balance'].min()-1, df['balance'].max()]),
    showlegend=False
)

fig.show()
